In [1]:
import pandas as pd
import yaml
import argparse
import torch
from model import YNet

In [2]:
%load_ext autoreload
%autoreload 2

#### Some hyperparameters and settings

In [3]:
CONFIG_FILE_PATH = 'config/sdd_longterm.yaml'  # yaml config file containing all the hyperparameters
DATASET_NAME = 'sdd'

TEST_DATA_PATH = 'data/SDD/test_longterm.pkl'
TEST_IMAGE_PATH = 'data/SDD/test'
OBS_LEN = 5  # in timesteps
PRED_LEN = 30  # in timesteps
NUM_GOALS = 20  # K_e
NUM_TRAJ = 1  # K_a

ROUNDS = 3  # Y-net is stochastic. How often to evaluate the whole dataset
BATCH_SIZE = 8

#### Load config file and print hyperparameters

In [4]:
with open(CONFIG_FILE_PATH) as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
experiment_name = CONFIG_FILE_PATH.split('.yaml')[0].split('config/')[1]
params

{'resize': 0.25,
 'batch_size': 8,
 'viz_epoch': 10,
 'learning_rate': 0.0001,
 'num_epochs': 300,
 'encoder_channels': [32, 32, 64, 64, 64],
 'decoder_channels': [64, 64, 64, 32, 32],
 'waypoints': [14, 29],
 'temperature': 1.8,
 'segmentation_model_fp': 'segmentation_models/SDD_segmentation.pth',
 'semantic_classes': 6,
 'loss_scale': 1000,
 'kernlen': 31,
 'nsig': 4,
 'use_features_only': False,
 'unfreeze': 100,
 'use_TTST': True,
 'rel_threshold': 0.002,
 'use_CWS': True,
 'CWS_params': {'sigma_factor': 6, 'ratio': 2, 'rot': True}}

#### Load preprocessed Data

In [5]:
df_test = pd.read_pickle(TEST_DATA_PATH)

In [6]:
df_test.head()

,trackId,frame,x,y,sceneId,metaId
0,5,311,1608.0,326.5,coupa_0,0
1,5,341,1608.0,326.5,coupa_0,0
2,5,371,1608.0,326.5,coupa_0,0
3,5,401,1608.0,326.5,coupa_0,0
4,5,431,1608.0,326.5,coupa_0,0


#### Initiate model and load pretrained weights

In [7]:
model = YNet(obs_len=OBS_LEN, pred_len=PRED_LEN, params=params)

In [8]:
model.load(f'pretrained_models/{experiment_name}_weights.pt')

<All keys matched successfully>


#### Evaluate model

In [9]:
model.evaluate(df_test, params, image_path=TEST_IMAGE_PATH,
               batch_size=BATCH_SIZE, rounds=ROUNDS, 
               num_goals=NUM_GOALS, num_traj=NUM_TRAJ, device=None, dataset_name=DATASET_NAME)

Preprocess data


Round:   0%|                                                                                     | 0/3 [00:00<?, ?it/s]

Start testing


Round:  33%|█████████████████████████▎                                                  | 1/3 [09:19<18:39, 559.50s/it]

Round 0: 
Test ADE: 46.825557708740234 
Test FDE: 64.77117156982422


Round:  67%|██████████████████████████████████████████████████▋                         | 2/3 [18:53<09:23, 563.81s/it]

Round 1: 
Test ADE: 46.7724723815918 
Test FDE: 65.5625991821289


Round: 100%|████████████████████████████████████████████████████████████████████████████| 3/3 [28:27<00:00, 569.01s/it]

Round 2: 
Test ADE: 47.00978469848633 
Test FDE: 66.36463928222656


Average performance over 3 rounds: 
Test ADE: 46.86927159627279 
Test FDE: 65.56613667805989
